# Modelos listos para submission

ziono...

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, PowerTransformer, OneHotEncoder
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
import pandas as pd
import re

In [2]:
def custom_features(dataframe_in):
    df = dataframe_in.copy(deep=True)

    df['month'] = pd.to_datetime(df['release_date']).dt.month
    df['release_date'] = pd.to_datetime(df['release_date']).apply(lambda x: x.to_julian_date())

    df['revenue'] = pd.Series([0 for _ in range(len(dataframe_in))])

    df.loc[df.publisher.str.match('.*microsoft.*', flags=re.IGNORECASE).values, 'revenue'] = 10.260
    df.loc[df.publisher.str.match('.*netease.*', flags=re.IGNORECASE).values, 'revenue'] = 6.668
    df.loc[df.publisher.str.match('.*activision.*', flags=re.IGNORECASE).values, 'revenue'] = 6.388
    df.loc[df.publisher.str.match('.*electronic.*', flags=re.IGNORECASE).values, 'revenue'] = 5.537
    df.loc[df.publisher.str.match('.*bandai.*', flags=re.IGNORECASE).values, 'revenue'] = 3.018
    df.loc[df.publisher.str.match('.*square.*', flags=re.IGNORECASE).values, 'revenue'] = 2.386
    df.loc[df.publisher.str.match('.*nexon.*', flags=re.IGNORECASE).values, 'revenue'] = 2.286
    df.loc[df.publisher.str.match('.*ubisoft.*', flags=re.IGNORECASE).values, 'revenue'] = 1.446
    df.loc[df.publisher.str.match('.*konami.*', flags=re.IGNORECASE).values, 'revenue'] = 1.303
    df.loc[df.publisher.str.match('.*SEGA.*').values, 'revenue'] = 1.153
    df.loc[df.publisher.str.match('.*capcom.*', flags=re.IGNORECASE).values, 'revenue'] = 0.7673
    df.loc[df.publisher.str.match('.*warner.*', flags=re.IGNORECASE).values, 'revenue'] = 0.7324

    return df

In [3]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, PowerTransformer, OneHotEncoder


class CategoriesTokenizer:
    def __init__(self):
        pass

    def __call__(self, doc):
        return doc.split(';')


boc_some_values = CountVectorizer(
    tokenizer = CategoriesTokenizer(),
    max_df = 1.0,
    min_df = 0.05  # hiperparametro a optimizar
    # valores para GridSearch : [5%, 10%, 15%] ???
    )


boc_many_values = CountVectorizer(
    tokenizer = CategoriesTokenizer(),
    max_df = 1.0,
    min_df = 1  # hiperparametro a optimizar
    # valores para GridSearch : [5, 10, 15] ???
    )


preprocesisng = ColumnTransformer(
    transformers=[
        ('BoC-plat',boc_some_values,'platforms'),
        ('BoC-cat',boc_some_values,'categories'),
        ('BoC-genres',boc_some_values,'genres'),
        ('BoC-tags',boc_some_values,'tags'),

        ('BoC-dev',boc_many_values,'developer'),
        ('BoC-pub',boc_many_values,'publisher'),

        ('OneHotEncoder',OneHotEncoder(handle_unknown='ignore'),['month']),
        ('MinMaxScaler',MinMaxScaler(),['required_age','price','release_date']),
        ('BoxCox',PowerTransformer(method='yeo-johnson'),['achievements','average_playtime','revenue']),
])

In [4]:
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectPercentile, f_classif

pipe = Pipeline([
    ('Pre-procesamiento',preprocesisng),
    ("selector", SelectPercentile(f_classif, percentile=95)),
    ('Clasificador',MLPClassifier(early_stopping =True,max_iter = 100, random_state=0))
])

In [5]:
from sklearn.model_selection import train_test_split

df_train = pd.read_pickle('train.pickle')
df_train = custom_features(df_train)
X_train, X_eval, y_train, y_eval = train_test_split(df_train, df_train['rating'], test_size=0.3, random_state=0, stratify=df_train['rating'])

In [6]:
from sklearn.metrics import classification_report

print("Resultados clasificación MLP")
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_eval)
print(classification_report(y_eval,y_pred))

Resultados clasificación MLP
                 precision    recall  f1-score   support

          Mixed       0.31      0.28      0.29       497
Mostly Positive       0.28      0.29      0.28       512
       Negative       0.46      0.33      0.38       387
       Positive       0.33      0.42      0.37       610
  Very Positive       0.41      0.36      0.38       359

       accuracy                           0.34      2365
      macro avg       0.36      0.34      0.34      2365
   weighted avg       0.35      0.34      0.34      2365



Entrenando con todos los datos

In [7]:
pipe_clf = Pipeline([
    ('Pre-procesamiento',preprocesisng),
    ("selector", SelectPercentile(f_classif, percentile=95)),
    ('Clasificador',MLPClassifier(early_stopping =True,max_iter = 100, random_state=0))
])

pipe_clf.fit(df_train, df_train['rating'])

Pipeline(steps=[('Pre-procesamiento',
                 ColumnTransformer(transformers=[('BoC-plat',
                                                  CountVectorizer(min_df=0.05,
                                                                  tokenizer=<__main__.CategoriesTokenizer object at 0x7fa6b77477f0>),
                                                  'platforms'),
                                                 ('BoC-cat',
                                                  CountVectorizer(min_df=0.05,
                                                                  tokenizer=<__main__.CategoriesTokenizer object at 0x7fa6b77477f0>),
                                                  'categories'),
                                                 ('BoC-genres',
                                                  CountVectorizer(min_df=0.05,
                                                                  token...
                                                  'publisher'),
                                                 ('OneHotEncoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['month']),
                                                 ('MinMaxScaler',
                                                  MinMaxScaler(),
                                                  ['required_age', 'price',
                                                   'release_date']),
                                                 ('BoxCox', PowerTransformer(),
                                                  ['achievements',
                                                   'average_playtime',
                                                   'revenue'])])),
                ('selector', SelectPercentile(percentile=95)),
                ('Clasificador',
                 MLPClassifier(early_stopping=True, max_iter=100,
                               random_state=0))])

## Regresión

In [8]:
import numpy as np
np.seterr(divide='ignore', invalid='ignore');

In [9]:
df_train = pd.read_pickle('train.pickle')
df_train = custom_features(df_train)
X_train, X_eval, y_train, y_eval = train_test_split(df_train, df_train['estimated_sells'], test_size=0.3, random_state=0)

In [10]:
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import r2_score, mean_squared_error

In [11]:
pipe = Pipeline([
    ('Pre-procesamiento',preprocesisng),
    ("selector", SelectPercentile(f_classif, percentile=95)),
    ('Regresor',BaggingRegressor(random_state=0))
])

print("Resultados regresión sin texto\n")

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_eval)

print("Error cuadrático medio = {}".format(mean_squared_error(y_eval,y_pred)))
print("Score R2 = {}".format(r2_score(y_eval,y_pred)))

Resultados regresión sin texto

Error cuadrático medio = 730686091203.4044
Score R2 = 0.5924228283913218


In [12]:
pipe_reg = Pipeline([
    ('Pre-procesamiento',preprocesisng),
    ("selector", SelectPercentile(f_classif, percentile=95)),
    ('Regresor',BaggingRegressor(random_state=0))
])

pipe_reg.fit(df_train, df_train['estimated_sells'])

Pipeline(steps=[('Pre-procesamiento',
                 ColumnTransformer(transformers=[('BoC-plat',
                                                  CountVectorizer(min_df=0.05,
                                                                  tokenizer=<__main__.CategoriesTokenizer object at 0x7fa6b77477f0>),
                                                  'platforms'),
                                                 ('BoC-cat',
                                                  CountVectorizer(min_df=0.05,
                                                                  tokenizer=<__main__.CategoriesTokenizer object at 0x7fa6b77477f0>),
                                                  'categories'),
                                                 ('BoC-genres',
                                                  CountVectorizer(min_df=0.05,
                                                                  token...
                                                  CountVectorizer(tokenizer=<__main__.CategoriesTokenizer object at 0x7fa6b7745c00>),
                                                  'publisher'),
                                                 ('OneHotEncoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['month']),
                                                 ('MinMaxScaler',
                                                  MinMaxScaler(),
                                                  ['required_age', 'price',
                                                   'release_date']),
                                                 ('BoxCox', PowerTransformer(),
                                                  ['achievements',
                                                   'average_playtime',
                                                   'revenue'])])),
                ('selector', SelectPercentile(percentile=95)),
                ('Regresor', BaggingRegressor(random_state=0))])

### Generando archivo de submission

In [13]:
from zipfile import ZipFile
import os

def generateFiles(predict_data, clf_pipe, rgr_pipe):
    """Genera los archivos a subir en CodaLab

    Input
    predict_data: Dataframe con los datos de entrada a predecir
    clf_pipe: pipeline del clf
    rgr_pipe: pipeline del rgr

    Ouput
    archivo de txt
    """
    y_pred_clf = clf_pipe.predict(predict_data)
    y_pred_rgr = rgr_pipe.predict(predict_data)
    
    with open('./predictions_clf.txt', 'w') as f:
        for item in y_pred_clf:
            f.write("%s\n" % item)

    with open('./predictions_rgr.txt', 'w') as f:
        for item in y_pred_rgr:
            f.write("%s\n" % item)

    with ZipFile('predictions.zip', 'w') as zipObj2:
       zipObj2.write('predictions_rgr.txt')
       zipObj2.write('predictions_clf.txt')

    os.remove("predictions_rgr.txt")
    os.remove("predictions_clf.txt")

In [16]:
df_test = pd.read_pickle('test.pickle')
df_test = custom_features(df_test)

In [17]:
generateFiles(df_test,pipe_clf,pipe_reg)